In [1]:
#Variabilil globali
width = 256
height = 256
mean_pre_trained =[0.485, 0.456, 0.406]
std_pre_trained =[0.229, 0.224, 0.225]
num_classes = 153
training = True

# TODO:
- Far partire Visdom
- Data agumentation
- Aumentare il knn con data agumentation.
- Allenare con nostre foto.



# Domande al prof:
- Perchè cambiando le dimensioni, aumentandole e diminuendole, l'accuracy cambia anche di molto?

# Import necessari

In [2]:
import torchvision.models as models
from torch.utils.data.dataset import Dataset
from PIL import Image
from os import path as pathFunction
from torchvision import transforms
import numpy as np
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier as KNN
from torch.autograd import Variable
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from torch.optim import SGD
from torch.autograd import Variable
from torchnet.logger import VisdomPlotLogger, VisdomSaver
from torchnet.meter import AverageValueMeter
from pathlib import Path


# Classe per la gestione del dataset

In [3]:
np.random.seed(1234)
torch.random.manual_seed(1234);

class ScenesDataset(Dataset):
    def __init__(self,base_path,txt_list,transform=None):
        #conserviamo il path alla cartella contenente le immagini
        self.base_path=base_path
        #carichiamo la lista dei file
        #sarà una matrice con n righe (numero di immagini) e 2 colonne (path, etichetta)
        self.images = np.loadtxt(txt_list,dtype=str,delimiter=',')
        #print("self.images ha i seguenti elementi:", len(self.images))
        #conserviamo il riferimento alla trasformazione da applicare
        self.transform = transform
    def __getitem__(self, index):
        #print("Get item numero -->", index)
        #recuperiamo il path dell'immagine di indice index e la relativa etichetta
        f,c = self.images[index]
        #carichiamo l'immagine utilizzando PIL e facciamo il resize a 3 canali.
        im = Image.open(self.base_path + "/" + f).convert("RGB")
        
        #Resize:
        im = im.resize((width,height))
        #se la trasfromazione è definita, applichiamola all'immagine
        if self.transform is not None:
            im = self.transform(im)       
        
        #convertiamo l'etichetta in un intero
        label = int(c)
        #restituiamo un dizionario contenente immagine etichetta
        #print("Mentre creo il tutto, label vale-->", label, ", name vale -->", f)
        return {'image' : im, 'label':label, 'name': f}
    #restituisce il numero di campioni: la lunghezza della lista "images"
    def __len__(self):
        #print("Ho invocato len, vale-->", len(self.images))
        return len(self.images)

# Funzione che ci restituisce media e varianza di un dataset.
- Prende in input il dataset.

In [4]:
def get_mean_devst(dataset):
    m = np.zeros(3)
    for sample in dataset:
        m+= np.array(sample['image'].sum(1).sum(1)) #accumuliamo la somma dei pixel canale per canale
    #dividiamo per il numero di immagini moltiplicato per il numero di pixel
    m=m/(len(dataset)*width*height)
    #procedura simile per calcolare la deviazione standard
    s = np.zeros(3)
    for sample in dataset:
        s+= np.array(((sample['image']-torch.Tensor(m).view(3,1,1))**2).sum(1).sum(1))
    s=np.sqrt(s/(len(dataset)*width*height))
    #print("Medie",m)
    #print("Dev.Std.",s)
    return m, s

# Prendiamo media e varianza per normalizzare successivamente i dati

In [5]:
#dataset = ScenesDataset('Dataset_base','train.txt',transform=transforms.ToTensor())
#m, s = get_mean_devst(dataset)

# Funzione per calcolare l'accuracy.

In [6]:
# Prende in input l'array di feature e il classificatore(knn.) 
def accuracy(classifier, samples):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    TOT = len(samples)
    for i in range(len(samples)):
        pred_label = classifier.predict(samples[i]["feature"].cpu().numpy().reshape(1, -1))
        if pred_label[0] == samples[i]["label"]:
            TP += 1
        else:
            TN += 1
        
    return float(TP)/len(samples)

In [7]:
def get_pred_label_and_target_label(classifier, samples):
    pred_label = []
    target_label = []
    for i in range(len(samples)):
        pred_label.append(classifier.predict(samples[i]["feature"].cpu().numpy().reshape(1, -1))[0])
        target_label.append(samples[i]["label"])
    return pred_label, target_label

# Funzione per estrazione feature:
- In input dataset (dataloader) e rete.

In [8]:
def extract_features(dataset, net):
    #Presa ogni riga del dataloader li passa alla net senza attivare il layer di classificazione
    feature_dataset = []
    print("Avviato extract_feature.")
    for i, dataset_train in enumerate(dataset):
        print(i, end= " ")
        x=Variable(dataset_train['image'], requires_grad=False)
        y=Variable(dataset_train['label'])
        x, y = x.cpu(), y.cpu()
        if torch.cuda.is_available():
            x, y = x.cuda(), y.cuda()
            #print("Con cuda")
        output = net(x)
        #print(output.grad, type(output.grad))

        #print("output len-->", len(output[0]), type(output), output[0], " - ", output[0].detach())
        feature_dataset.append({"label": dataset_train['label'], "feature":output[0].detach(), "name": dataset_train['name']})

    return feature_dataset

# Funzione per creare l'input per l'oggetto dataframe:
- In input dataset (dataloader) e rete.

In [9]:
def get_dataframe(dataset, net, feature_dataset = None):
    print("Avviato get_dataframe.")
    if feature_dataset == None:
        feature_dataset = extract_features(dataset, net) 
    print("\nHo concluso extract_features, sto creare feature_dataset_matrix")
    feature_dataset_matrix = np.zeros((len(feature_dataset), len(feature_dataset[0]["feature"])))    
    #Qui abbiamo nelle righe tutte le immagini, nella lable feature tutte le 9000 colonne, ossia le feature.
    label_array = np.zeros(len(feature_dataset))
    for i in range(0, len(feature_dataset)):#302
        print(i, end= " ")
        for j in range(0, len(feature_dataset[0]["feature"])):#153
            if j == 0:#salviamo la y finale nell'array label_array
                label_array[i] = feature_dataset[i]['label']
                #print(i, end= " ")
            feature_dataset_matrix[i][j] =feature_dataset[i]["feature"][j] 
        #Elimino l'elemento per cercare di risparmiare un pò di memoria.
        #feature_dataset.pop(i)
    return feature_dataset_matrix, label_array

In [98]:

def train_classification(model, train_loader, test_loader, lr=0.001, epochs=20, momentum=0.8, exp_name = 'experiment' ):
    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(),lr, momentum=momentum)
    loaders = {'train':train_loader, 'test':test_loader}
    losses = {'train':[], 'test':[]}
    accuracies = {'train':[], 'test':[]}
    
    
    loss_meter = AverageValueMeter()
    acc_meter = AverageValueMeter()
    loss_logger = VisdomPlotLogger('line', env=exp_name, opts={'title': 'Loss', 'legend':['train','test']})
    acc_logger = VisdomPlotLogger('line', env=exp_name, opts={'title': 'Accuracy','legend':['train','test']})
    visdom_saver = VisdomSaver(envs=[exp_name])

    if torch.cuda.is_available():
        model=model.cuda()

    for e in range(epochs):
        #print("Primo ciclo for.")
        for mode in ['train', 'test']:
            #print("Secondo ciclo for.")
            
            loss_meter.reset()
            acc_meter.reset()
            
            if mode=='train':
                model.train()
            else:
                model.eval()
            epoch_loss = 0
            epoch_acc = 0
            samples = 0
            #print("Mode-->",mode)
            #print("Enumerate-->", loaders[mode])
            for i, batch in enumerate(loaders[mode]):
                #trasformiamo i tensori in variabili
                x=Variable(batch['image'], requires_grad=(mode=='train'))
                y=Variable(batch['label'])
                if torch.cuda.is_available():
                    x, y = x.cuda(), y.cuda()
                    #print("Con cuda")
                #else:
                    #print("Senza cuda")
                output = model(x)
                #print(type(output))
                l = criterion(output,y)
                if mode=='train':
                    l.backward()
                    optimizer.step()
                    optimizer.zero_grad()
                #print("L-->",l.item())
                acc = accuracy_score(y.cpu().data,output.cpu().max(1)[1].data)
                epoch_loss+=l.data.item()*x.shape[0]
                epoch_acc+=acc*x.shape[0]
                samples+=x.shape[0]
                '''print ("\r[%s] Epoch %d/%d. Iteration %d/%d. Loss: %0.2f. Accuracy: %0.2f\t\t\t\t\t" % \
                (mode, e+1, epochs, i, len(loaders[mode]), epoch_loss/samples, epoch_acc/samples),
                epoch_loss/samples,
                epoch_acc/samples,
                losses[mode].append(epoch_loss))'''
                accuracies[mode].append(epoch_acc)
                n = batch['image'].shape[0]
                loss_meter.add(l.item()*n,n)
                acc_meter.add(acc*n,n)
                #loss_logger.log(e+(i+1)/len(loaders[mode]), loss_meter.value()[0], name=mode)
                #acc_logger.log(e+(i+1)/len(loaders[mode]), acc_meter.value()[0], name=mode)


            loss_logger.log(e+1, loss_meter.value()[0], name=mode)
            acc_logger.log(e+1, acc_meter.value()[0], name=mode)
            if mode == "train":
                print(e, end = " ")
            #print("Fine secondo ciclo for")
        '''print("\r[%s] Epoch %d/%d. Iteration %d/%d. Loss: %0.2f. Accuracy: %0.2f\t\t\t\t\t" % \
        (mode, e+1, epochs, i, len(loaders[mode]), epoch_loss, epoch_acc))
        torch.save(model.state_dict(), "./" + exp_name + ".pth")
        print("Pesi aggiornati.")'''
        torch.save(model.state_dict(), "./" + exp_name + ".pth")

    #print("Ho finito.")
    #restituiamo il modello e i vari log
    return model, (losses, accuracies)

In [11]:
def test_model_classification(model, test_loader):
    softmax = nn.Softmax(dim=1)
    model.eval()
    model.cpu()
    preds = []
    gts = []
    for batch in test_loader:
        x=Variable(batch["image"])
        x = x.cpu()
        #applichiamo la funzione softmax per avere delle probabilità
        if torch.cuda.is_available():
            x = x.cuda()
            model.cuda()
        pred = softmax(model(x)).data.cpu().numpy().copy()
        gt = batch["label"].cpu().numpy().copy()
        #print("Pred-->", pred, ", gt-->", gt)
        preds.append(pred)
        gts.append(gt)
        #print(len(preds), len(gts))
    return np.concatenate(preds),np.concatenate(gts)

In [12]:
def test_model_classification_our_network(model, our_model, test_loader):
    softmax = nn.Softmax(dim=1)
    model.eval()
    model.cpu()
    our_model.eval()
    our_model.cpu()
    preds = []
    gts = []
    for batch in test_loader:
        x=Variable(batch["image"])
        x = x.cpu()
        #applichiamo la funzione softmax per avere delle probabilità
        if torch.cuda.is_available():
            x = x.cuda()
            model.cuda()
            our_model.cuda()
            
        pred = softmax(our_model(model(x))).data.cpu().numpy().copy()
        gt = batch["label"].cpu().numpy().copy()
        #print("Pred-->", pred, ", gt-->", gt)
        preds.append(pred)
        gts.append(gt)
        #print(len(preds), len(gts))
    return np.concatenate(preds),np.concatenate(gts)

In [13]:
def extract_features_our_network(dataset, net):
    #Presa ogni riga del dataloader li passa alla net senza attivare il layer di classificazione
    feature_dataset = []
    print("Avviato extract_feature.")
    for i, dataset_train in enumerate(dataset):
        print(i, end= " ")
        x=Variable(dataset_train["feature"], requires_grad=False)
        x = x.cpu()
        if torch.cuda.is_available():
            x = x.cuda()
        output = net(x)
        feature_dataset.append({"feature": output[0].detach(), "label":dataset_train["label"]})
    return feature_dataset

# Creiamo Oggetto Dataset e dataloader, sia base che data augmentation.

In [ ]:
transformss = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean_pre_trained,std_pre_trained)])
barilla_train = ScenesDataset('Dataset_base','train.txt',transform=transformss)
barilla_test = ScenesDataset('Dataset_base','test.txt',transform=transformss)
barilla_train_loader = torch.utils.data.DataLoader(barilla_train, batch_size=10, num_workers=0, shuffle=True)
barilla_test_loader = torch.utils.data.DataLoader(barilla_test, batch_size=10, num_workers=0)

In [ ]:
transformss = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean_pre_trained,std_pre_trained)])
barilla_train_da = ScenesDataset('Dataset_data_augmentation','train_data_augmentation.txt',transform=transformss)
barilla_test_da = ScenesDataset('Dataset_data_augmentation','test_data_augmentation.txt',transform=transformss)
barilla_train_loader_da = torch.utils.data.DataLoader(barilla_train_da, batch_size=10, num_workers=0, shuffle=True)
barilla_test_loader_da = torch.utils.data.DataLoader(barilla_test_da, batch_size=10, num_workers=0)

# Allenamento dataset first level data augmentation - Rialleniamo alexnet con 20 epoche.

In [43]:
'''transformss = transforms.Compose([transforms.ToTensor()])
#Prendo la media e deviazione standard e normalizzo il trainset
dataset = ScenesDataset('new_image_first_level','train_first_level.txt',transform=transformss)
mean_pre_trained,std_pre_trained = get_mean_devst(dataset)
transformss = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean_pre_trained,std_pre_trained)])
barilla_train_fl = ScenesDataset('new_image_first_level','train_first_level.txt',transform=transformss)
#Prendo la media e deviazione standard e normalizzo il testset
dataset = ScenesDataset('new_image_first_level','test_first_level.txt',transform=transformss)
mean_pre_trained,std_pre_trained = get_mean_devst(dataset)
transformss = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean_pre_trained,std_pre_trained)])
barilla_test_fl = ScenesDataset('new_image_first_level','test_first_level.txt',transform=transformss)
#Creo i dataloader
barilla_train_loader_fl = torch.utils.data.DataLoader(barilla_train_fl, batch_size=10, num_workers=0, shuffle=True)
barilla_test_loader_fl = torch.utils.data.DataLoader(barilla_test_fl, batch_size=10, num_workers=0)'''

mean_pre_trained =[0.485, 0.456, 0.406]
std_pre_trained =[0.229, 0.224, 0.225]
transformss = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean_pre_trained,std_pre_trained)])
barilla_train = ScenesDataset('Data_augmentation_new_image_first_level','Data_augmentation_train_first_level.txt',transform=transformss)
barilla_test = ScenesDataset('Data_augmentation_new_image_first_level','Data_augmentation_test_first_level.txt',transform=transformss)
barilla_train_loader_fl = torch.utils.data.DataLoader(barilla_train, batch_size=10, num_workers=0, shuffle=True)
barilla_test_loader_fl = torch.utils.data.DataLoader(barilla_test, batch_size=10, num_workers=0)

In [44]:
net = models.alexnet(pretrained=True)
net.classifier[6] = nn.Linear(4096, 17) #Numero esatto di classi nel nostro dataset.
sum([p.numel() for p in net.parameters()])
name_save = "Data_augmentation_retraining_alexnet_17classes_newDataset"

In [45]:
path = "./" + name_save + ".pth"
my_file = Path(path)
training = False
if training == False and my_file.is_file():#Carico lo state altrimenti rialleno
    net.load_state_dict(torch.load(path))
    print("Caricato")
else:
    #net.load_state_dict(torch.load(path))
    lenet_mnist, lenet_mnist_logs = train_classification(net, epochs=20, train_loader = barilla_train_loader_fl,
                                                         test_loader = barilla_test_loader_fl, exp_name = name_save,
                                                         lr = 0.001, momentum = 0.8)
    torch.save(net.state_dict(), "./" + name_save + ".pth")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 

# Accuracy senza usare il knn

In [46]:
lenet_mnist_predictions, lenet_mnist_gt = test_model_classification(net, barilla_train_loader_fl)
print ("Accuracy aLexNet di train su barillatestloader: %0.2f" % \
accuracy_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))
print("Precision_score-->")
print(precision_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
print("Recall_score-->")
print(recall_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
print("F1_Score-->")
print(f1_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
print("Matrice di confusione-->\n")
print(confusion_matrix(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))

Accuracy aLexNet di train su barillatestloader: 1.00
Precision_score-->
1.0
Recall_score-->
1.0
F1_Score-->
1.0
Matrice di confusione-->

[[ 64   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0  80   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0 120   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0  96   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0  32   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0  96   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  48   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0 752   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0 128   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0  96   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0  64   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0 192   0   0   0   0   0]
 [  0   0 

In [49]:
lenet_mnist_predictions, lenet_mnist_gt = test_model_classification(net, barilla_test_loader_fl)
print ("Accuracy aLexNet di test su barillatestloader: %0.2f" % \
accuracy_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))
print("Precision_score-->")
print(precision_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
print("Recall_score-->")
print(recall_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
print("F1_Score-->")
print(f1_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
print("Matrice di confusione-->\n")
print(confusion_matrix(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))

Accuracy aLexNet di test su barillatestloader: 1.00
Precision_score-->
0.9995343783951576
Recall_score-->
0.9944852941176471
F1_Score-->
0.9968733036586689
Matrice di confusione-->

[[ 32   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0  40   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0  64   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0  48   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0  16   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0  48   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  24   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0 376   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  64   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0  48   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   3   0   0  29   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0

# Creiamo il dataframe e il knn

In [ ]:
knn_1 = KNN(n_neighbors=1)
knn_3 = KNN(n_neighbors=3)
knn_5 = KNN(n_neighbors=5)


barilla_train_loader_OB_da = torch.utils.data.DataLoader(barilla_train_fl, batch_size=1, num_workers=0, shuffle=True)
barilla_test_loader_OB_da = torch.utils.data.DataLoader(barilla_test_fl, batch_size=1, num_workers=0)


if torch.cuda.is_available():
    net = net.cuda()
    torch.cuda.empty_cache()
net.eval()

input_for_datafram_train, label_array_train = get_dataframe(barilla_train_loader_OB_da, net)
df = pd.DataFrame(input_for_datafram_train)
knn_1.fit(df, label_array_train)
knn_3.fit(df, label_array_train)
knn_5.fit(df, label_array_train)

In [ ]:
feature_test = extract_features(barilla_test_loader_OB_da, net)
print("Accuracy con rete preallenata e dataset base.")
print("**************************1NN**************************")
pred_label, target_label = get_pred_label_and_target_label(knn_1, feature_test)
print("Accuracy-->", accuracy_score(target_label,pred_label))
print("Precision_score-->", precision_score(target_label,pred_label, average = 'macro'))
print("Recall_score-->", recall_score(target_label,pred_label, average = 'macro'))
print("F1_Score-->", f1_score(target_label,pred_label, average = 'macro'))
print("Matrice di confusione-->\n")
print(confusion_matrix(target_label,pred_label))

print("**************************3NN**************************")
pred_label, target_label = get_pred_label_and_target_label(knn_3, feature_test)
print("Accuracy-->", accuracy_score(target_label,pred_label))
print("Precision_score-->", precision_score(target_label,pred_label, average = 'macro'))
print("Recall_score-->", recall_score(target_label,pred_label, average = 'macro'))
print("F1_Score-->", f1_score(target_label,pred_label, average = 'macro'))
print("Matrice di confusione-->\n")
print(confusion_matrix(target_label,pred_label))

print("**************************5NN**************************")
pred_label, target_label = get_pred_label_and_target_label(knn_5, feature_test)
print("Accuracy-->", accuracy_score(target_label,pred_label))
print("Precision_score-->", precision_score(target_label,pred_label, average = 'macro'))
print("Recall_score-->", recall_score(target_label,pred_label, average = 'macro'))
print("F1_Score-->", f1_score(target_label,pred_label, average = 'macro'))
print("Matrice di confusione-->\n")
print(confusion_matrix(target_label,pred_label))


# -----------------------------------------------------------------------------------------------

# Allenamento dataset first level  - Rialleniamo alexnet con 20 epoche.

In [50]:
'''transformss = transforms.Compose([transforms.ToTensor()])
#Prendo la media e deviazione standard e normalizzo il trainset
dataset = ScenesDataset('new_image_first_level','train_first_level.txt',transform=transformss)
mean_pre_trained,std_pre_trained = get_mean_devst(dataset)
transformss = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean_pre_trained,std_pre_trained)])
barilla_train_fl = ScenesDataset('new_image_first_level','train_first_level.txt',transform=transformss)
#Prendo la media e deviazione standard e normalizzo il testset
dataset = ScenesDataset('new_image_first_level','test_first_level.txt',transform=transformss)
mean_pre_trained,std_pre_trained = get_mean_devst(dataset)
transformss = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean_pre_trained,std_pre_trained)])
barilla_test_fl = ScenesDataset('new_image_first_level','test_first_level.txt',transform=transformss)
#Creo i dataloader
barilla_train_loader_fl = torch.utils.data.DataLoader(barilla_train_fl, batch_size=10, num_workers=0, shuffle=True)
barilla_test_loader_fl = torch.utils.data.DataLoader(barilla_test_fl, batch_size=10, num_workers=0)'''

mean_pre_trained =[0.485, 0.456, 0.406]
std_pre_trained =[0.229, 0.224, 0.225]
transformss = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean_pre_trained,std_pre_trained)])
barilla_train = ScenesDataset('new_image_first_level','test_first_level.txt',transform=transformss)
barilla_test = ScenesDataset('new_image_first_level','test_first_level.txt',transform=transformss)
barilla_train_loader_fl = torch.utils.data.DataLoader(barilla_train, batch_size=10, num_workers=0, shuffle=True)
barilla_test_loader_fl = torch.utils.data.DataLoader(barilla_test, batch_size=10, num_workers=0)

In [51]:
net = models.alexnet(pretrained=True)
net.classifier[6] = nn.Linear(4096, 17) #Numero esatto di classi nel nostro dataset.
sum([p.numel() for p in net.parameters()])
name_save = "retraining_alexnet_17classes_newDataset"

In [52]:
path = "./" + name_save + ".pth"
my_file = Path(path)
training = True
if training == False and my_file.is_file():#Carico lo state altrimenti rialleno
    net.load_state_dict(torch.load(path))
    print("Caricato")
else:
    #net.load_state_dict(torch.load(path))
    lenet_mnist, lenet_mnist_logs = train_classification(net, epochs=40, train_loader = barilla_train_loader_fl,
                                                         test_loader = barilla_test_loader_fl, exp_name = name_save,
                                                         lr = 0.0001, momentum = 0.8)
    torch.save(net.state_dict(), "./" + name_save + ".pth")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 

# Accuracy senza usare il knn

In [47]:
lenet_mnist_predictions, lenet_mnist_gt = test_model_classification(net, barilla_train_loader_fl)
print ("Accuracy aLexNet di train su barillatestloader: %0.2f" % \
accuracy_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))
print("Precision_score-->")
print(precision_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
print("Recall_score-->")
print(recall_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
print("F1_Score-->")
print(f1_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
print("Matrice di confusione-->\n")
print(confusion_matrix(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))

Accuracy aLexNet di train su barillatestloader: 1.00
Precision_score-->
1.0
Recall_score-->
1.0
F1_Score-->
1.0
Matrice di confusione-->

[[ 64   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0  80   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0 120   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0  96   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0  32   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0  96   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  48   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0 752   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0 128   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0  96   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0  64   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0 192   0   0   0   0   0]
 [  0   0 

In [48]:
lenet_mnist_predictions, lenet_mnist_gt = test_model_classification(net, barilla_test_loader_fl)
print ("Accuracy aLexNet di test su barillatestloader: %0.2f" % \
accuracy_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))
print("Precision_score-->")
print(precision_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
print("Recall_score-->")
print(recall_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
print("F1_Score-->")
print(f1_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
print("Matrice di confusione-->\n")
print(confusion_matrix(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))

Accuracy aLexNet di test su barillatestloader: 1.00
Precision_score-->
0.9995343783951576
Recall_score-->
0.9944852941176471
F1_Score-->
0.9968733036586689
Matrice di confusione-->

[[ 32   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0  40   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0  64   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0  48   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0  16   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0  48   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  24   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0 376   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  64   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0  48   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   3   0   0  29   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0

# Creiamo il dataframe e il knn

In [ ]:
knn_1 = KNN(n_neighbors=1)
knn_3 = KNN(n_neighbors=3)
knn_5 = KNN(n_neighbors=5)


barilla_train_loader_OB_da = torch.utils.data.DataLoader(barilla_train_fl, batch_size=1, num_workers=0, shuffle=True)
barilla_test_loader_OB_da = torch.utils.data.DataLoader(barilla_test_fl, batch_size=1, num_workers=0)


if torch.cuda.is_available():
    net = net.cuda()
    torch.cuda.empty_cache()
net.eval()

input_for_datafram_train, label_array_train = get_dataframe(barilla_train_loader_OB_da, net)
df = pd.DataFrame(input_for_datafram_train)
knn_1.fit(df, label_array_train)
knn_3.fit(df, label_array_train)
knn_5.fit(df, label_array_train)

In [ ]:
feature_test = extract_features(barilla_test_loader_OB_da, net)
print("Accuracy con rete preallenata e dataset base.")
print("**************************1NN**************************")
pred_label, target_label = get_pred_label_and_target_label(knn_1, feature_test)
print("Accuracy-->", accuracy_score(target_label,pred_label))
print("Precision_score-->", precision_score(target_label,pred_label, average = 'macro'))
print("Recall_score-->", recall_score(target_label,pred_label, average = 'macro'))
print("F1_Score-->", f1_score(target_label,pred_label, average = 'macro'))
print("Matrice di confusione-->\n")
print(confusion_matrix(target_label,pred_label))

print("**************************3NN**************************")
pred_label, target_label = get_pred_label_and_target_label(knn_3, feature_test)
print("Accuracy-->", accuracy_score(target_label,pred_label))
print("Precision_score-->", precision_score(target_label,pred_label, average = 'macro'))
print("Recall_score-->", recall_score(target_label,pred_label, average = 'macro'))
print("F1_Score-->", f1_score(target_label,pred_label, average = 'macro'))
print("Matrice di confusione-->\n")
print(confusion_matrix(target_label,pred_label))

print("**************************5NN**************************")
pred_label, target_label = get_pred_label_and_target_label(knn_5, feature_test)
print("Accuracy-->", accuracy_score(target_label,pred_label))
print("Precision_score-->", precision_score(target_label,pred_label, average = 'macro'))
print("Recall_score-->", recall_score(target_label,pred_label, average = 'macro'))
print("F1_Score-->", f1_score(target_label,pred_label, average = 'macro'))
print("Matrice di confusione-->\n")
print(confusion_matrix(target_label,pred_label))


# -----------------------------------------------------------------------------------------------

# Fine Pasta_piccolini  -----------------------------------------------------------------------<br><br>
# Inizio Pasta_special_pack -------------------------------------------------------------------

In [ ]:
transformss = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean_pre_trained,std_pre_trained)])
barilla_train = ScenesDataset('single_dataset/Pasta_special_pack','single_dataset/Pasta_special_pack/train.txt',transform=transformss)
barilla_test = ScenesDataset('single_dataset/Pasta_special_pack','single_dataset/Pasta_special_pack/test.txt',transform=transformss)
barilla_train_loader = torch.utils.data.DataLoader(barilla_train, batch_size=10, num_workers=0, shuffle=True)
barilla_test_loader = torch.utils.data.DataLoader(barilla_test, batch_size=10, num_workers=0)

In [ ]:
net = models.squeezenet1_0(pretrained=True)
#Update last layer for squeezenet1_0
net.classifier[1] = nn.Conv2d(512, 4, kernel_size=(1,1), stride=(1,1))

sum([p.numel() for p in net.parameters()])
name_save = "retraining_squeezenet1_0_4classes_dataset_Pasta_special_pack"

In [ ]:
path = "./" + name_save + ".pth"
my_file = Path(path)
training = False
if training == False and my_file.is_file():#Carico lo state altrimenti rialleno
    net.load_state_dict(torch.load(path))
    print("Caricato")
else:
    #net.load_state_dict(torch.load(path))
    lenet_mnist, lenet_mnist_logs = train_classification(net, epochs=60, train_loader = barilla_train_loader,
                                                         test_loader = barilla_test_loader, exp_name = name_save, lr = 0.0001, momentum = 0.9)
    torch.save(net.state_dict(), "./" + name_save + ".pth")

# Accuracy senza usare il knn

In [ ]:
lenet_mnist_predictions, lenet_mnist_gt = test_model_classification(net, barilla_train_loader)
print ("Accuracy aLexNet di train su barillatestloader: %0.2f" % \
accuracy_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))
print("Precision_score-->")
print(precision_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
print("Recall_score-->")
print(recall_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
print("F1_Score-->")
print(f1_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
print("Matrice di confusione-->\n")
print(confusion_matrix(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))

In [ ]:
lenet_mnist_predictions, lenet_mnist_gt = test_model_classification(net, barilla_test_loader)
print ("Accuracy aLexNet di test su barillatestloader: %0.2f" % \
accuracy_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))
print("Precision_score-->")
print(precision_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
print("Recall_score-->")
print(recall_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
print("F1_Score-->")
print(f1_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
print("Matrice di confusione-->\n")
print(confusion_matrix(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))

# Creiamo il dataframe e il knn

In [ ]:
knn_1 = KNN(n_neighbors=1)
knn_3 = KNN(n_neighbors=3)
knn_5 = KNN(n_neighbors=5)


barilla_train_loader_OB = torch.utils.data.DataLoader(barilla_train, batch_size=1, num_workers=0, shuffle=True)
barilla_test_loader_OB = torch.utils.data.DataLoader(barilla_test, batch_size=1, num_workers=0)


if torch.cuda.is_available():
    net = net.cuda()
    torch.cuda.empty_cache()
net.eval()

input_for_datafram_train, label_array_train = get_dataframe(barilla_train_loader_OB, net)
df = pd.DataFrame(input_for_datafram_train)
knn_1.fit(df, label_array_train)
knn_3.fit(df, label_array_train)
knn_5.fit(df, label_array_train)

In [ ]:
feature_test = extract_features(barilla_test_loader_OB, net)
print("Accuracy con rete preallenata e dataset base.")
print("**************************1NN**************************")
pred_label, target_label = get_pred_label_and_target_label(knn_1, feature_test)
print("Accuracy-->", accuracy_score(target_label,pred_label))
print("Precision_score-->", precision_score(target_label,pred_label, average = 'macro'))
print("Recall_score-->", recall_score(target_label,pred_label, average = 'macro'))
print("F1_Score-->", f1_score(target_label,pred_label, average = 'macro'))
print("Matrice di confusione-->\n")
print(confusion_matrix(target_label,pred_label))

print("**************************3NN**************************")
pred_label, target_label = get_pred_label_and_target_label(knn_3, feature_test)
print("Accuracy-->", accuracy_score(target_label,pred_label))
print("Precision_score-->", precision_score(target_label,pred_label, average = 'macro'))
print("Recall_score-->", recall_score(target_label,pred_label, average = 'macro'))
print("F1_Score-->", f1_score(target_label,pred_label, average = 'macro'))
print("Matrice di confusione-->\n")
print(confusion_matrix(target_label,pred_label))

print("**************************5NN**************************")
pred_label, target_label = get_pred_label_and_target_label(knn_5, feature_test)
print("Accuracy-->", accuracy_score(target_label,pred_label))
print("Precision_score-->", precision_score(target_label,pred_label, average = 'macro'))
print("Recall_score-->", recall_score(target_label,pred_label, average = 'macro'))
print("F1_Score-->", f1_score(target_label,pred_label, average = 'macro'))
print("Matrice di confusione-->\n")
print(confusion_matrix(target_label,pred_label))


In [ ]:
feature_test = extract_features(barilla_test_loader_OB, net)
print("Accuracy con rete preallenata e dataset base.")
print("**************************1NN**************************")
pred_label, target_label = get_pred_label_and_target_label(knn_1, feature_test)
print("Accuracy-->", accuracy_score(target_label,pred_label))
print("Precision_score-->", precision_score(target_label,pred_label, average = 'macro'))
print("Recall_score-->", recall_score(target_label,pred_label, average = 'macro'))
print("F1_Score-->", f1_score(target_label,pred_label, average = 'macro'))
print("Matrice di confusione-->\n")
print(confusion_matrix(target_label,pred_label))

print("**************************3NN**************************")
pred_label, target_label = get_pred_label_and_target_label(knn_3, feature_test)
print("Accuracy-->", accuracy_score(target_label,pred_label))
print("Precision_score-->", precision_score(target_label,pred_label, average = 'macro'))
print("Recall_score-->", recall_score(target_label,pred_label, average = 'macro'))
print("F1_Score-->", f1_score(target_label,pred_label, average = 'macro'))
print("Matrice di confusione-->\n")
print(confusion_matrix(target_label,pred_label))

print("**************************5NN**************************")
pred_label, target_label = get_pred_label_and_target_label(knn_5, feature_test)
print("Accuracy-->", accuracy_score(target_label,pred_label))
print("Precision_score-->", precision_score(target_label,pred_label, average = 'macro'))
print("Recall_score-->", recall_score(target_label,pred_label, average = 'macro'))
print("F1_Score-->", f1_score(target_label,pred_label, average = 'macro'))
print("Matrice di confusione-->\n")
print(confusion_matrix(target_label,pred_label))


In [70]:
def get_dataloader(path, txt_file, train = True, batch_size = 10):
    mean_pre_trained =[0.485, 0.456, 0.406]
    std_pre_trained =[0.229, 0.224, 0.225]
    transformss = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean_pre_trained,std_pre_trained)])
    barilla_dataset = ScenesDataset(path, txt_file, transform=transformss)
    if train == True:
        dataloader = torch.utils.data.DataLoader(barilla_dataset, batch_size, num_workers=0, shuffle=True)
    else: 
        dataloader = torch.utils.data.DataLoader(barilla_dataset, batch_size, num_workers=0)
    return dataloader


In [71]:
def get_scenedataset(path, txt_file, train = True, batch_size = 1):
    mean_pre_trained =[0.485, 0.456, 0.406]
    std_pre_trained =[0.229, 0.224, 0.225]
    transformss = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean_pre_trained,std_pre_trained)])
    barilla_dataset = ScenesDataset(path, txt_file, transform=transformss)
    return barilla_dataset


In [72]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [96]:
dictionary = {  'Bio': 4,
                'Cereali': 5,              
                'Farina_semola_pizza': 8,
                'Legumi': 6,
                'Pasta_5_Cereali': 3,
                'Pasta_all\'uovo_ripiena': 6,
                'Pasta_Base': 18,
                'Pasta_box': 47,
                'Pasta_emiliane_chef': 2,    
                'Pasta_integrale': 8,
                'Pasta_integrale_voiello': 2,
                'Pasta_piccolini': 6,              
                'Pasta_special_pack': 4,
                'Pasta_specialita': 15,
                'PASTA_UOVO_5_CEREALI': 2,
                'senza_glutine': 9,
                'Sughi': 7,
             }



In [86]:
class TreeNet:
    def __init__(self, dictionary, data_augmentation = ""):
        self.package_net = models.alexnet(pretrained=True)
        self.package_net.classifier[6] = nn.Linear(4096, 17) #Numero esatto di classi nel nostro dataset.
        #self.package_net = models.alexnet(num_classes = 17)
        self.softmax = nn.Softmax(dim=1)
        self.load_dict(self.package_net, data_augmentation + "retraining_alexnet_17classes_newDataset")
        print("Parametri caricati")
        self.dictionary = dictionary
        
    def load_dict(self, net, name):
        net.load_state_dict(torch.load("./" + name + ".pth"))
        
    def print_accuracy(self):
        mean_pre_trained =[0.485, 0.456, 0.406]
        std_pre_trained =[0.229, 0.224, 0.225]
        transformss = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean_pre_trained,std_pre_trained)])
        barilla_test = ScenesDataset('new_image_first_level','test_first_level.txt',transform=transformss)
        barilla_test_loader_fl = torch.utils.data.DataLoader(barilla_test, batch_size=10, num_workers=0)
        lenet_mnist_predictions, lenet_mnist_gt = test_model_classification(self.package_net, barilla_test_loader_fl)
        print ("Accuracy aLexNet di train su barillatestloader: %0.2f" % \
        accuracy_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))
    
    
    def get_class(self, net, img, n_classes, key = "", data_augmentation = ""):
        #print(n_classes, key)
        net.eval()
        net.cpu()

        if key == "":
            mean_pre_trained =[0.485, 0.456, 0.406]
            std_pre_trained =[0.229, 0.224, 0.225]
        else:
            path = data_augmentation + 'single_dataset/' + key
            txt_file = path + '/train.txt'
            #mean_pre_trained =[0.485, 0.456, 0.406]
            #std_pre_trained =[0.229, 0.224, 0.225]
            #transformss = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean_pre_trained, std_pre_trained)])
            #dataset = ScenesDataset(path, txt_file, transform=transformss)
            #mean_pre_trained,std_pre_trained = get_mean_devst(dataset)
        
        mean_pre_trained =[0.485, 0.456, 0.406]
        std_pre_trained =[0.229, 0.224, 0.225]
        
        img = img.resize((256,256))
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean_pre_trained, std_pre_trained)])
        img = transform(img)
        x = Variable(img).cpu()
        x = x.unsqueeze(0)
        
        #Carico la giusta net in base al parametro key. Se diverso da vuoto allora devo caricare i giusti parametri e crearne
        #una al volo
        if key != "":
            net = models.squeezenet1_0(pretrained=True)
            #Update last layer for squeezenet1_0
            net.classifier[1] = nn.Conv2d(512, n_classes, kernel_size=(1,1), stride=(1,1))
            name_save = data_augmentation + "retraining_squeezenet_"+ str(n_classes) +"classes_dataset_" + key
            #Alleno o carico lo stato attuale se train è false.
            path = "./" + name_save + ".pth"
            net.load_state_dict(torch.load(path))
        
        pred = self.softmax(net(x)).data.cpu().numpy().copy()
        #print("Prima --> ", pred)
        pred = pred.argmax(1)
        #print(pred , key)
        #Caso di get_pastabox
        if key == "":
            filepath = "./classes_first_level.txt"
        else:
            filepath = data_augmentation + 'single_dataset/' + key + '/classes.txt'
        with open(filepath, "r+") as fp:
            for i in range(0, n_classes):
                line = fp.readline()
                n, c = line.strip().split(",")

                if int(c) == int(pred):
                    #print("name: ", n, " c: ", c)
                    name = n
        # Pred ci darà una classe, attraverso il file classes_first_level prendiamo il nome e lo utilizziamo per prendere
        # la nuova predizione.
        
        #pred abbiamo ciò che ha predetto.
        
        #Ci serve ciò che era realmente
        '''if key == "":
            filepath = "./classes_first_level.txt"
        else:
            filepath = 'single_dataset/' + key + '/test.txt'
        with open(filepath, "r+") as fp:
            for i in range(len(open(filepath, "r+").readlines())):
                line = fp.readline()
                n, c = line.strip().split(",")

                if int(c) == int(pred):
                    print("name: ", n, " c: ", c)
                    name = n
        '''
        return name, pred

    

In [87]:
bn = TreeNet(dictionary, "Data_augmentation_")
#bn.print_accuracy()

Parametri caricati


In [76]:
img = Image.open("./new_image_first_level/Mealsolution_Barilla_Bio_Farinabiotipo21000g_Front.png").convert("RGB")
#img = Image.open("./pasta_specialita_2.jpg").convert("RGB")
name, pred = bn.get_class(bn.package_net, img, 17)
print(name, pred)
if name == "Pasta_Integrale":
    name = "Pasta_integrale"
if name == "Pasta_Integrale_voiello":
    name = "Pasta_integrale_voiello"
if name == "Pasta_specialitÃ\xa0":
    name = "Pasta_specialita"
bn.get_class(bn.package_net, img, dictionary[name], name)

Bio [0]


('Farinabiotipo21000g', array([0], dtype=int64))

In [88]:
def get_original_class(data_augmentation, name_img):    
    name_list = name_img.split("_")
    #print(name_list[-2], name_list[-3])
    return name_list[-2], name_list[-3]

    

In [97]:
data_augmentation = True

if data_augmentation == True:
    path = 'Data_augmentation_new_image_first_level/'
else: 
    path = 'new_image_first_level/'
    
entries = os.listdir(path)

TP = 0
error = 0
total = 0
accuracy = []
t = []

if data_augmentation == False:
    filepath = "train_first_level.txt"
else:
    filepath = "Data_augmentation_train_first_level.txt"
    
with open(filepath, "r+") as fp:
    for i in range(len(open(filepath, "r+").readlines())):
        line = fp.readline()
        entry, c = line.strip().split(",")
        total += 1
        #Apro l'immagine
        #print("\nImmagine-->", entry)
        img = Image.open(path + entry).convert("RGB")
        if data_augmentation == True:
            name, pred = bn.get_class(bn.package_net, img, 17, data_augmentation ="Data_augmentation_")
        else:
            name, pred = bn.get_class(bn.package_net, img, 17)
        #print(name, dictionary[name])
        if name == "Pasta_Integrale":
            name = "Pasta_integrale"
        if name == "Pasta_Integrale_voiello":
            name = "Pasta_integrale_voiello"
        if name == "Pasta_specialitÃ\xa0":
            name = "Pasta_specialita"
        if data_augmentation == True:
            final_name, final_pred = bn.get_class(bn.package_net, img, dictionary[name], name, data_augmentation ="Data_augmentation_")
        else:
            final_name, final_pred = bn.get_class(bn.package_net, img, dictionary[name], name)

        original_name, original_pred = get_original_class(data_augmentation, entry)
        #print("Predizione package: ", name, ", predizione classe finale: ", final_name, ", classe finale reale: ", original_name)
        if final_name == original_name:
            TP += 1
        else:
            error += 1
        try:
            accuracy.append(TP/total)
            t.append(total)
            print("Ciclo n°: ", total, ", accuracy: ", TP/total)
        except:
            pass
    print("Totale accuracy di train-->", str(total), ", accuracy: ", str(TP/total))



TP = 0
error = 0
total = 0
accuracy = []
t = []

    
if data_augmentation == False:
    filepath = "test_first_level.txt"
else:
    filepath = "Data_augmentation_test_first_level.txt"
    
with open(filepath, "r+") as fp:
    for i in range(len(open(filepath, "r+").readlines())):
        line = fp.readline()
        entry, c = line.strip().split(",")
        total += 1
        #Apro l'immagine
        #print("\nImmagine-->", entry)
        img = Image.open(path + entry).convert("RGB")
        if data_augmentation == True:
            name, pred = bn.get_class(bn.package_net, img, 17, data_augmentation ="Data_augmentation_")
        else:
            name, pred = bn.get_class(bn.package_net, img, 17)
        #print(dictionary[name])
        if name == "Pasta_Integrale":
            name = "Pasta_integrale"
        if name == "Pasta_Integrale_voiello":
            name = "Pasta_integrale_voiello"
        if name == "Pasta_specialitÃ\xa0":
            name = "Pasta_specialita"
        if data_augmentation == True:
            final_name, final_pred = bn.get_class(bn.package_net, img, dictionary[name], name, data_augmentation ="Data_augmentation_")
        else:
            final_name, final_pred = bn.get_class(bn.package_net, img, dictionary[name], name)

        original_name, original_pred = get_original_class(data_augmentation, entry)
        #print("Predizione package: ", name, ", predizione classe finale: ", final_name, ", classe finale reale: ", original_name)
        
        if final_name == original_name:
            TP += 1
        else:
            error += 1
        try:
            accuracy.append(TP/total)
            t.append(total)
            print("Ciclo n°: ", total, ", accuracy: ", TP/total)
        except:
            pass
    print("Totale accuracy di train-->", str(total), ", accuracy: ", str(TP/total))

Ciclo n°:  1 , accuracy:  1.0
Ciclo n°:  2 , accuracy:  1.0
Ciclo n°:  3 , accuracy:  1.0
Ciclo n°:  4 , accuracy:  1.0
Ciclo n°:  5 , accuracy:  1.0
Ciclo n°:  6 , accuracy:  1.0
Ciclo n°:  7 , accuracy:  1.0
Ciclo n°:  8 , accuracy:  1.0
Ciclo n°:  9 , accuracy:  1.0
Ciclo n°:  10 , accuracy:  1.0
Ciclo n°:  11 , accuracy:  1.0
Ciclo n°:  12 , accuracy:  1.0
Ciclo n°:  13 , accuracy:  1.0
Ciclo n°:  14 , accuracy:  1.0
Ciclo n°:  15 , accuracy:  1.0
Ciclo n°:  16 , accuracy:  1.0
Ciclo n°:  17 , accuracy:  1.0
Ciclo n°:  18 , accuracy:  1.0
Ciclo n°:  19 , accuracy:  1.0
Ciclo n°:  20 , accuracy:  1.0
Ciclo n°:  21 , accuracy:  1.0
Ciclo n°:  22 , accuracy:  1.0
Ciclo n°:  23 , accuracy:  1.0
Ciclo n°:  24 , accuracy:  1.0
Ciclo n°:  25 , accuracy:  1.0
Ciclo n°:  26 , accuracy:  1.0
Ciclo n°:  27 , accuracy:  1.0
Ciclo n°:  28 , accuracy:  1.0
Ciclo n°:  29 , accuracy:  1.0
Ciclo n°:  30 , accuracy:  1.0
Ciclo n°:  31 , accuracy:  1.0
Ciclo n°:  32 , accuracy:  1.0
Ciclo n°:  33 , a

Ciclo n°:  201 , accuracy:  0.8905472636815921
Ciclo n°:  202 , accuracy:  0.8910891089108911
Ciclo n°:  203 , accuracy:  0.8916256157635468
Ciclo n°:  204 , accuracy:  0.8921568627450981
Ciclo n°:  205 , accuracy:  0.8926829268292683
Ciclo n°:  206 , accuracy:  0.8932038834951457
Ciclo n°:  207 , accuracy:  0.893719806763285
Ciclo n°:  208 , accuracy:  0.8942307692307693
Ciclo n°:  209 , accuracy:  0.8947368421052632
Ciclo n°:  210 , accuracy:  0.8952380952380953
Ciclo n°:  211 , accuracy:  0.8957345971563981
Ciclo n°:  212 , accuracy:  0.8962264150943396
Ciclo n°:  213 , accuracy:  0.8967136150234741
Ciclo n°:  214 , accuracy:  0.897196261682243
Ciclo n°:  215 , accuracy:  0.8976744186046511
Ciclo n°:  216 , accuracy:  0.8981481481481481
Ciclo n°:  217 , accuracy:  0.8986175115207373
Ciclo n°:  218 , accuracy:  0.8990825688073395
Ciclo n°:  219 , accuracy:  0.8995433789954338
Ciclo n°:  220 , accuracy:  0.9
Ciclo n°:  221 , accuracy:  0.9004524886877828
Ciclo n°:  222 , accuracy:  0.

Ciclo n°:  377 , accuracy:  0.9336870026525199
Ciclo n°:  378 , accuracy:  0.9312169312169312
Ciclo n°:  379 , accuracy:  0.9287598944591029
Ciclo n°:  380 , accuracy:  0.9263157894736842
Ciclo n°:  381 , accuracy:  0.9238845144356955
Ciclo n°:  382 , accuracy:  0.9240837696335078
Ciclo n°:  383 , accuracy:  0.9216710182767625
Ciclo n°:  384 , accuracy:  0.9192708333333334
Ciclo n°:  385 , accuracy:  0.9168831168831169
Ciclo n°:  386 , accuracy:  0.9145077720207254
Ciclo n°:  387 , accuracy:  0.9121447028423773
Ciclo n°:  388 , accuracy:  0.9097938144329897
Ciclo n°:  389 , accuracy:  0.910025706940874
Ciclo n°:  390 , accuracy:  0.9076923076923077
Ciclo n°:  391 , accuracy:  0.9053708439897699
Ciclo n°:  392 , accuracy:  0.9030612244897959
Ciclo n°:  393 , accuracy:  0.9033078880407125
Ciclo n°:  394 , accuracy:  0.9035532994923858
Ciclo n°:  395 , accuracy:  0.9037974683544304
Ciclo n°:  396 , accuracy:  0.9040404040404041
Ciclo n°:  397 , accuracy:  0.9042821158690176
Ciclo n°:  398

Ciclo n°:  552 , accuracy:  0.9311594202898551
Ciclo n°:  553 , accuracy:  0.9312839059674503
Ciclo n°:  554 , accuracy:  0.9314079422382672
Ciclo n°:  555 , accuracy:  0.9315315315315316
Ciclo n°:  556 , accuracy:  0.9316546762589928
Ciclo n°:  557 , accuracy:  0.9317773788150808
Ciclo n°:  558 , accuracy:  0.931899641577061
Ciclo n°:  559 , accuracy:  0.9320214669051878
Ciclo n°:  560 , accuracy:  0.9321428571428572
Ciclo n°:  561 , accuracy:  0.9322638146167558
Ciclo n°:  562 , accuracy:  0.9323843416370107
Ciclo n°:  563 , accuracy:  0.9325044404973357
Ciclo n°:  564 , accuracy:  0.9326241134751773
Ciclo n°:  565 , accuracy:  0.9327433628318584
Ciclo n°:  566 , accuracy:  0.9328621908127208
Ciclo n°:  567 , accuracy:  0.9329805996472663
Ciclo n°:  568 , accuracy:  0.9330985915492958
Ciclo n°:  569 , accuracy:  0.9332161687170475
Ciclo n°:  570 , accuracy:  0.9333333333333333
Ciclo n°:  571 , accuracy:  0.9334500875656743
Ciclo n°:  572 , accuracy:  0.9335664335664335
Ciclo n°:  573

Ciclo n°:  727 , accuracy:  0.9477303988995873
Ciclo n°:  728 , accuracy:  0.9478021978021978
Ciclo n°:  729 , accuracy:  0.9478737997256516
Ciclo n°:  730 , accuracy:  0.947945205479452
Ciclo n°:  731 , accuracy:  0.948016415868673
Ciclo n°:  732 , accuracy:  0.9480874316939891
Ciclo n°:  733 , accuracy:  0.9481582537517054
Ciclo n°:  734 , accuracy:  0.9482288828337875
Ciclo n°:  735 , accuracy:  0.9482993197278912
Ciclo n°:  736 , accuracy:  0.9483695652173914
Ciclo n°:  737 , accuracy:  0.9484396200814111
Ciclo n°:  738 , accuracy:  0.948509485094851
Ciclo n°:  739 , accuracy:  0.9485791610284168
Ciclo n°:  740 , accuracy:  0.9486486486486486
Ciclo n°:  741 , accuracy:  0.9487179487179487
Ciclo n°:  742 , accuracy:  0.9487870619946092
Ciclo n°:  743 , accuracy:  0.9488559892328399
Ciclo n°:  744 , accuracy:  0.9489247311827957
Ciclo n°:  745 , accuracy:  0.9489932885906041
Ciclo n°:  746 , accuracy:  0.9490616621983914
Ciclo n°:  747 , accuracy:  0.9491298527443106
Ciclo n°:  748 ,

...

Ciclo n°:  392 , accuracy:  0.9260204081632653
Ciclo n°:  393 , accuracy:  0.926208651399491
Ciclo n°:  394 , accuracy:  0.9263959390862944
Ciclo n°:  395 , accuracy:  0.9265822784810127
Ciclo n°:  396 , accuracy:  0.9267676767676768
Ciclo n°:  397 , accuracy:  0.9269521410579346
Ciclo n°:  398 , accuracy:  0.9271356783919598
Ciclo n°:  399 , accuracy:  0.9273182957393483
Ciclo n°:  400 , accuracy:  0.9275
Ciclo n°:  401 , accuracy:  0.9276807980049875
Ciclo n°:  402 , accuracy:  0.927860696517413
Ciclo n°:  403 , accuracy:  0.9280397022332506
Ciclo n°:  404 , accuracy:  0.9282178217821783
Ciclo n°:  405 , accuracy:  0.928395061728395
Ciclo n°:  406 , accuracy:  0.9285714285714286
Ciclo n°:  407 , accuracy:  0.9287469287469288
Ciclo n°:  408 , accuracy:  0.928921568627451
Ciclo n°:  409 , accuracy:  0.9290953545232273
Ciclo n°:  410 , accuracy:  0.9292682926829269
Ciclo n°:  411 , accuracy:  0.9294403892944039
Ciclo n°:  412 , accuracy:  0.9296116504854369
Ciclo n°:  413 , accuracy:  0

Ciclo n°:  567 , accuracy:  0.9029982363315696
Ciclo n°:  568 , accuracy:  0.903169014084507
Ciclo n°:  569 , accuracy:  0.9033391915641477
Ciclo n°:  570 , accuracy:  0.9035087719298246
Ciclo n°:  571 , accuracy:  0.9036777583187391
Ciclo n°:  572 , accuracy:  0.9038461538461539
Ciclo n°:  573 , accuracy:  0.9022687609075044
Ciclo n°:  574 , accuracy:  0.9024390243902439
Ciclo n°:  575 , accuracy:  0.9008695652173913
Ciclo n°:  576 , accuracy:  0.9010416666666666
Ciclo n°:  577 , accuracy:  0.8994800693240901
Ciclo n°:  578 , accuracy:  0.8996539792387543
Ciclo n°:  579 , accuracy:  0.8998272884283247
Ciclo n°:  580 , accuracy:  0.9
Ciclo n°:  581 , accuracy:  0.9001721170395869
Ciclo n°:  582 , accuracy:  0.9003436426116839
Ciclo n°:  583 , accuracy:  0.8987993138936535
Ciclo n°:  584 , accuracy:  0.898972602739726
Ciclo n°:  585 , accuracy:  0.8991452991452992
Ciclo n°:  586 , accuracy:  0.8993174061433447
Ciclo n°:  587 , accuracy:  0.899488926746167
Ciclo n°:  588 , accuracy:  0.8

Ciclo n°:  743 , accuracy:  0.892328398384926
Ciclo n°:  744 , accuracy:  0.8924731182795699
Ciclo n°:  745 , accuracy:  0.8926174496644296
Ciclo n°:  746 , accuracy:  0.8927613941018767
Ciclo n°:  747 , accuracy:  0.892904953145917
Ciclo n°:  748 , accuracy:  0.893048128342246
Ciclo n°:  749 , accuracy:  0.8931909212283045
Ciclo n°:  750 , accuracy:  0.8933333333333333
Ciclo n°:  751 , accuracy:  0.8934753661784287
Ciclo n°:  752 , accuracy:  0.8936170212765957
Ciclo n°:  753 , accuracy:  0.8937583001328021
Ciclo n°:  754 , accuracy:  0.8938992042440318
Ciclo n°:  755 , accuracy:  0.8940397350993378
Ciclo n°:  756 , accuracy:  0.8941798941798942
Ciclo n°:  757 , accuracy:  0.8943196829590488
Ciclo n°:  758 , accuracy:  0.8944591029023746
Ciclo n°:  759 , accuracy:  0.8945981554677207
Ciclo n°:  760 , accuracy:  0.8947368421052632
Ciclo n°:  761 , accuracy:  0.8948751642575559
Ciclo n°:  762 , accuracy:  0.89501312335958
Ciclo n°:  763 , accuracy:  0.8951507208387942
Ciclo n°:  764 , a

Ciclo n°:  918 , accuracy:  0.8812636165577342
Ciclo n°:  919 , accuracy:  0.8813928182807399
Ciclo n°:  920 , accuracy:  0.8815217391304347
Ciclo n°:  921 , accuracy:  0.8816503800217155
Ciclo n°:  922 , accuracy:  0.8817787418655098
Ciclo n°:  923 , accuracy:  0.8819068255687974
Ciclo n°:  924 , accuracy:  0.8820346320346321
Ciclo n°:  925 , accuracy:  0.8821621621621621
Ciclo n°:  926 , accuracy:  0.8822894168466523
Ciclo n°:  927 , accuracy:  0.8824163969795038
Ciclo n°:  928 , accuracy:  0.8825431034482759
Ciclo n°:  929 , accuracy:  0.8826695371367062
Ciclo n°:  930 , accuracy:  0.8827956989247312
Ciclo n°:  931 , accuracy:  0.882921589688507
Ciclo n°:  932 , accuracy:  0.8830472103004292
Ciclo n°:  933 , accuracy:  0.8831725616291533
Ciclo n°:  934 , accuracy:  0.8832976445396146
Ciclo n°:  935 , accuracy:  0.8834224598930481
Ciclo n°:  936 , accuracy:  0.8835470085470085
Ciclo n°:  937 , accuracy:  0.8836712913553896
Ciclo n°:  938 , accuracy:  0.8837953091684435
Ciclo n°:  939

Ciclo n°:  1091 , accuracy:  0.8955087076076994
Ciclo n°:  1092 , accuracy:  0.8956043956043956
Ciclo n°:  1093 , accuracy:  0.8956999085086916
Ciclo n°:  1094 , accuracy:  0.8957952468007313
Ciclo n°:  1095 , accuracy:  0.8958904109589041
Ciclo n°:  1096 , accuracy:  0.8959854014598541
Ciclo n°:  1097 , accuracy:  0.8960802187784868
Ciclo n°:  1098 , accuracy:  0.8961748633879781
Ciclo n°:  1099 , accuracy:  0.8962693357597816
Ciclo n°:  1100 , accuracy:  0.8963636363636364
Ciclo n°:  1101 , accuracy:  0.896457765667575
Ciclo n°:  1102 , accuracy:  0.896551724137931
Ciclo n°:  1103 , accuracy:  0.8966455122393472
Ciclo n°:  1104 , accuracy:  0.8967391304347826
Ciclo n°:  1105 , accuracy:  0.8968325791855204
Ciclo n°:  1106 , accuracy:  0.8969258589511754
Ciclo n°:  1107 , accuracy:  0.8970189701897019
Ciclo n°:  1108 , accuracy:  0.8971119133574007
Ciclo n°:  1109 , accuracy:  0.8972046889089269
Ciclo n°:  1110 , accuracy:  0.8972972972972973
Ciclo n°:  1111 , accuracy:  0.89738973897

# Fare funzione che allena tutti gli n classificatori delle varie sottoclassi in un colpo con lo stesso lr e momentum, poi sistemare singolarmente solo quelli che non hanno restituito dei buoni risultati. Commentare il codice per la stampa ad ogni iterazione tanto vediamo l'andamento su visdom e farsi restituire/stampare solo le accuracy oltre che salvare gli stati dei vari classificatori.
# <br> Inoltre, Creare un dictionary/matrice con i nomi delle cartelle/numero di classi contenute per aiutare l'automazione del tutto. Poi cancellare tutti i dict e rilanciare il tutto.

# Fare in modo che la funzione prenda in input un bool che decide se  allenare o caricare gli state se li trova.

In [68]:
def training_all_classificator(dictionary, train = True, knn_indicator = False, all_indicator = False, pre_name = "", net_name = "retraining_squeezenet_"):    
    for key, num_classes in dictionary.items():
        print()
        print()
        print("Inizio nuovo allenamento sulla cartella " + key + " con un numero di classi totali: " + str(num_classes))
        
        #Creo i dataloader
        barilla_train_loader = get_dataloader(pre_name + 'single_dataset/' + key, pre_name + 'single_dataset/'+ key +'/train.txt')
        barilla_test_loader = get_dataloader(pre_name + 'single_dataset/' + key, pre_name + 'single_dataset/'+ key +'/test.txt', train = False)
        
        #Creo la squeeznet con i parametri preallenati, cambio l'ultimo layer e creo il nome per poi salvare lo stato di allenamento.
        net = models.squeezenet1_0(pretrained=True)
        #Update last layer for squeezenet1_0
        net.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        
        #net = models.alexnet(pretrained=True)
        #net.classifier[6] = nn.Linear(4096, num_classes) #Numero esatto di classi nel nostro dataset.
        
        
        print("TODO: Da togliere, numero di parametri: " + str(sum([p.numel() for p in net.parameters()])))
        name_save = pre_name +   net_name + str(num_classes) +"classes_dataset_"+key

        #Alleno o carico lo stato attuale se train è false.
        path = "./" + name_save + ".pth"
        my_file = Path(path)
        if train == False and my_file.is_file():#Carico lo state altrimenti rialleno
            net.load_state_dict(torch.load(path))
            print("Caricato lo stato per il classificatore " + name_save + ".")
        else:
            #net.load_state_dict(torch.load(path)) #Buoni parametri: lr = 0.0009, momentum = 0.8
            lenet_mnist, lenet_mnist_logs = train_classification(net, epochs=80, train_loader = barilla_train_loader,
                                                                 test_loader = barilla_test_loader,
                                                                 exp_name = name_save, lr = 0.0005, momentum = 0.6)
            print("Allenamento finito, stato salvato correttamente.")
            torch.save(net.state_dict(), "./" + name_save + ".pth")
            
            
        #Stampo accuracy ed eventualmente anche gli altri indicatori se richiesto.
        lenet_mnist_predictions, lenet_mnist_gt = test_model_classification(net, barilla_train_loader)
        print ("Accuracy aLexNet di train su "+ name_save +": %0.2f" % \
            accuracy_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))
        if all_indicator: 
            print("Precision_score-->")
            print(precision_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
            print("Recall_score-->")
            print(recall_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
            print("F1_Score-->")
            print(f1_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
            print("Matrice di confusione-->\n")
            print(confusion_matrix(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))
        
        lenet_mnist_predictions, lenet_mnist_gt = test_model_classification(net, barilla_test_loader)
        print ("Accuracy aLexNet di test su "+ name_save +": %0.2f" % \
            accuracy_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))
        if all_indicator: 
            print("Precision_score-->")
            print(precision_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
            print("Recall_score-->")
            print(recall_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
            print("F1_Score-->")
            print(f1_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1), average = 'macro'))
            print("Matrice di confusione-->\n")
            print(confusion_matrix(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))
        
        #Parte relativa al knn.
        if knn_indicator:
            #Creo i vari knn passandogli le feature estratte dalla rete allenata.
            knn_1 = KNN(n_neighbors=1)
            knn_3 = KNN(n_neighbors=3)
            knn_5 = KNN(n_neighbors=5)

            barilla_train = get_scenedataset(pre_name +'single_dataset/' + key, pre_name +'single_dataset/'+ key +'/train.txt')
            barilla_test = get_scenedataset(pre_name +'single_dataset/' + key, pre_name +'single_dataset/'+ key +'/test.txt')

            barilla_train_loader_OB = torch.utils.data.DataLoader(barilla_train, batch_size=1, num_workers=0, shuffle=True)
            barilla_test_loader_OB = torch.utils.data.DataLoader(barilla_test, batch_size=1, num_workers=0)


            if torch.cuda.is_available():
                net = net.cuda()
                torch.cuda.empty_cache()
            net.eval()

            input_for_datafram_train, label_array_train = get_dataframe(barilla_train_loader_OB, net)
            df = pd.DataFrame(input_for_datafram_train)
            knn_1.fit(df, label_array_train)
            knn_3.fit(df, label_array_train)
            knn_5.fit(df, label_array_train)
            
            #Estraggo le feature del test loader e calcolo accuracy e vari indicatori su tutto.
            feature_test = extract_features(barilla_test_loader_OB, net)
            print("**************************1NN**************************")
            pred_label, target_label = get_pred_label_and_target_label(knn_1, feature_test)
            print("Accuracy-->", accuracy_score(target_label,pred_label))
            if all_indicator:
                print("Precision_score-->", precision_score(target_label,pred_label, average = 'macro'))
                print("Recall_score-->", recall_score(target_label,pred_label, average = 'macro'))
                print("F1_Score-->", f1_score(target_label,pred_label, average = 'macro'))
                print("Matrice di confusione-->\n")
                print(confusion_matrix(target_label,pred_label))
            
            print("**************************3NN**************************")
            pred_label, target_label = get_pred_label_and_target_label(knn_3, feature_test)
            print("Accuracy-->", accuracy_score(target_label,pred_label))
            if all_indicator:
                print("Precision_score-->", precision_score(target_label,pred_label, average = 'macro'))
                print("Recall_score-->", recall_score(target_label,pred_label, average = 'macro'))
                print("F1_Score-->", f1_score(target_label,pred_label, average = 'macro'))
                print("Matrice di confusione-->\n")
                print(confusion_matrix(target_label,pred_label))
            
            if pre_name != "" and key != "Pasta_emiliane_chef": #Si rompe in questo caso perchè non abbiamo 5 esempi.
                print("**************************5NN**************************")
                pred_label, target_label = get_pred_label_and_target_label(knn_5, feature_test)
                print("Accuracy-->", accuracy_score(target_label,pred_label))
                if all_indicator:
                    print("Precision_score-->", precision_score(target_label,pred_label, average = 'macro'))
                    print("Recall_score-->", recall_score(target_label,pred_label, average = 'macro'))
                    print("F1_Score-->", f1_score(target_label,pred_label, average = 'macro'))
                    print("Matrice di confusione-->\n")
                    print(confusion_matrix(target_label,pred_label))

In [69]:
training_all_classificator(dictionary, True, False, False)



Inizio nuovo allenamento sulla cartella Pasta_integrale con un numero di classi totali: 8
TODO: Da togliere, numero di parametri: 739528


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 Allenamento finito, stato salvato correttamente.
Accuracy aLexNet di train su retraining_squeezenet_8classes_dataset_Pasta_integrale: 1.00
Accuracy aLexNet di test su retraining_squeezenet_8classes_dataset_Pasta_integrale: 0.75


In [94]:
training_all_classificator(dictionary, False, False, False, "Data_augmentation_")
#TODO: Da ripartire da pasta integrale, è successo qualcosa durante la costruzione, sistemare a mano.



Inizio nuovo allenamento sulla cartella Pasta_integrale_voiello con un numero di classi totali: 2
TODO: Da togliere, numero di parametri: 736450
Caricato lo stato per il classificatore Data_augmentation_retraining_squeezenet_2classes_dataset_Pasta_integrale_voiello.
Accuracy aLexNet di train su Data_augmentation_retraining_squeezenet_2classes_dataset_Pasta_integrale_voiello: 1.00
Accuracy aLexNet di test su Data_augmentation_retraining_squeezenet_2classes_dataset_Pasta_integrale_voiello: 1.00


In [93]:

dictionary = {  #'Cereali': 8,
                #'Pasta_all\'uovo_ripiena': 6,
                'Pasta_integrale_voiello': 2,
                #'PASTA_UOVO_5_CEREALI': 2,
                # 'Pasta_box': 47,
             }
